In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
train_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_df.pop('id')
X = train_df.drop(['id'], axis=1)
Y = X.pop('loss')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, 
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn import ensemble
from sklearn import metrics

# Input parameters for Gradient Boosting Regressor
params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls',
          'criterion': 'mse'}

grad_boost = ensemble.GradientBoostingRegressor(**params)
grad_boost.fit(X_scaled, y_train)

In [ ]:
predicted_loss = grad_boost.predict(X_test_scaled)

mse = metrics.mean_squared_error(y_test, predicted_loss)
print(f"The mean squared error (MSE): {mse}")


In [ ]:
# scaling test data
test_df = scaler.transform(test_df)

submission['loss'] = grad_boost.predict(test_df)
submission.to_csv('./grad_boost_submission.csv', index=False)
submission.head()